In [ ]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
import datetime as datetime

In [ ]:
# Data pre-processing
transform_train = transforms.Compose([transforms.Resize((70)), # Resizing the dataset samples to 70x70
                                transforms.RandomHorizontalFlip(), # Randomly flips the data samples horizontally
                                transforms.ToTensor(),
                                transforms.Normalize((0.5),(0.5))]) # Normalizing the dataset

transform_test = transforms.Compose([transforms.Resize((70)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5),(0.5))])

# Downloading and loading the dataset into the workspace
train = torchvision.datasets.MNIST(root='./data',train=True, transform=transform_train, download=True)
test = torchvision.datasets.MNIST(root='./data',train=False, transform=transform_test, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=32, shuffle=False)
cuda = torch.device('cuda') # to train the model on a GPU

In [ ]:
# Structure of the GoogleNet architecture
class Inception(nn.Module):
    def __init__(self,in_channel,c1,c2,c3,c4):
        super(Inception,self).__init__()
        self.norm1_1=nn.BatchNorm2d(in_channel,eps=1e-3)
        self.p1_1=nn.Conv2d(in_channels=in_channel,out_channels=c1,kernel_size=1)
        self.norm2_1 = nn.BatchNorm2d(in_channel, eps=1e-3)
        self.p2_1=nn.Conv2d(in_channels=in_channel,out_channels=c2[0],kernel_size=1)
        self.norm2_2 = nn.BatchNorm2d(c2[0], eps=1e-3)
        self.p2_2=nn.Conv2d(in_channels=c2[0],out_channels=c2[1],kernel_size=3,padding=1)
        self.norm3_1 = nn.BatchNorm2d(in_channel, eps=1e-3)
        self.p3_1=nn.Conv2d(in_channels=in_channel,out_channels=c3[0],kernel_size=1)
        self.norm3_2 = nn.BatchNorm2d(c3[0], eps=1e-3)
        self.p3_2=nn.Conv2d(in_channels=c3[0],out_channels=c3[1],kernel_size=5,padding=2)
        self.p4_1 = nn.MaxPool2d(kernel_size=3,stride=1,padding=1)
        self.norm4_2 = nn.BatchNorm2d(in_channel, eps=1e-3)
        self.p4_2 = nn.Conv2d(in_channels=in_channel, out_channels=c4, kernel_size=1)
 
    def forward(self, x):
        p1=self.p1_1(F.relu(self.norm1_1(x)))
        p2=self.p2_2(F.relu(self.norm2_2(self.p2_1(F.relu(self.norm2_1(x))))))
        p3=self.p3_2(F.relu(self.norm3_2(self.p3_1(F.relu(self.norm3_1(x))))))
        p4=self.p4_2(F.relu(self.norm4_2(self.p4_1(x))))
        return torch.cat((p1,p2,p3,p4),dim=1)

class GoogleNet(nn.Module):
    def __init__(self,in_channel,num_classes):
        super(GoogleNet,self).__init__()
        layers=[]
        layers+=[nn.Conv2d(in_channels=in_channel,out_channels=64,kernel_size=7,stride=2,padding=3),
                 nn.ReLU(),
                 nn.MaxPool2d(kernel_size=3,stride=2,padding=1)]
        layers+=[nn.Conv2d(in_channels=64,out_channels=64,kernel_size=1),
                 nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,padding=1),
                 nn.MaxPool2d(kernel_size=3,stride=2,padding=1)]
        layers+=[Inception(192,64,(96,128),(16,32),32),
                 Inception(256,128,(128,192),(32,96),64),
                 nn.MaxPool2d(kernel_size=3,stride=2,padding=1)]
        layers+=[Inception(480, 192, (96, 208), (16, 48), 64),
                 Inception(512, 160, (112, 224), (24, 64), 64),
                 Inception(512, 128, (128, 256), (24, 64), 64),
                 Inception(512, 112, (144, 288), (32, 64), 64),
                 Inception(528, 256, (160, 320), (32, 128), 128),
               nn.MaxPool2d(kernel_size=3, stride=2, padding=1)]
        layers += [Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AvgPool2d(kernel_size=2)]
        self.net = nn.Sequential(*layers)
        self.dense=nn.Linear(1024,num_classes)
 
 
    def forward(self,x):
        x=self.net(x)
        x=x.view(-1,1024*1*1)
        x=self.dense(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Using CUDA enabled GPU for training the model
model = GoogleNet(1, 10).to(device) # Setting the number of input classes = 1 and output classes = 10
print(model)

In [ ]:
loss = nn.CrossEntropyLoss() # Using Cross Entropy as Loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # Setting the optimizer for training the model
cost = 0
epochs = 5 # Setting the number of epochs for which the model is to be trained

iterations = []
train_losses = []
test_losses = []
train_acc = []
test_acc = []

# Training the model
for epoch in range(epochs):
    model.train()
    correct = 0
    for X, Y in train_loader:
        X = X.to(cuda)
        Y = Y.to(cuda)
        optimizer.zero_grad()
        hypo = model(X)
        cost = loss(hypo, Y)
        cost.backward()
        optimizer.step()
        prediction = hypo.data.max(1)[1]
        correct += prediction.eq(Y.data).sum()

    model.eval()
    correct2 = 0
    for data, target in test_loader:
        data = data.to(cuda)
        target = target.to(cuda)
        output = model(data)
        cost2 = loss(output, target)
        prediction = output.data.max(1)[1]
        correct2 += prediction.eq(target.data).sum()

    iterations.append(epoch)
    train_losses.append(cost.tolist())
    test_losses.append(cost2.tolist())
    train_acc.append((100*correct/len(train_loader.dataset)).tolist())
    test_acc.append((100*correct2/len(test_loader.dataset)).tolist())
    print("Epoch : {:>4} / cost : {:>.9}".format(epoch + 1, cost))
    print('Train set Accuracy: {:.2f}%'.format(100. * correct / len(train_loader.dataset)))
    print('Test set Accuracy: {:.2f}%'.format(100. * correct2 / len(test_loader.dataset)))
    timestamp = datetime.datetime.now()
    print("Date/Time stamp", timestamp)

In [ ]:
# Train Accuracy vs Validation accuracy plot
plt.figure(figsize=(10, 7))
plt.plot(train_acc, color='green', label='train accuracy')
plt.plot(test_acc, color='blue', label='validataion accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.show()

# Train Loss vs Validation Loss plot
plt.figure(figsize=(10, 7))
plt.plot(train_losses, color='orange', label='train loss')
plt.plot(test_losses, color='red', label='validataion loss')
plt.legend()
plt.savefig('loss.png')
plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Utilizng CUDA enabled GPU
print(device)
classes = ['0','1','2','3','4','5','6','7','8','9'] # Classes of the MNIST dataset

# Building the heatmap of the correctly classified and misclassified data samples
heatmap = pd.DataFrame(data=0,index=classes,columns=classes)
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(16):
            true_label = labels[i].item()
            predicted_label = predicted[i].item()
            heatmap.iloc[true_label,predicted_label] += 1
_, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(heatmap, annot=True, fmt='d',cmap='YlGnBu')
figure = ax.get_figure()    
figure.savefig('heatmap.png', dpi=400)
plt.show()

In [ ]:
# Saving the model
torch.save(model,'GoogleNet_CIFAR10.pth')